In [ ]:
# Assuming your kernel autoinstalled renv, uncomment the following code
# to install the rest of the packages
# renv::restore()

In [1]:
library(renv)
library(GGally)
library(tidymodels)
library(tidyverse)
library(leaps)
library(caret)
library(kknn)
library(randomForest)


Attaching package: 'renv'


The following objects are masked from 'package:stats':

    embed, update


The following objects are masked from 'package:utils':

    history, upgrade


The following objects are masked from 'package:base':

    autoload, load, remove


Loading required package: ggplot2

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2

── Attaching packages ────────────────────────────────────────────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5      ✔ rsample      1.2.0 
✔ dials        1.2.0      ✔ tibble       3.2.1 
✔ dplyr        1.1.4      ✔ tidyr        1.3.0 
✔ infer        1.0.6      ✔ tune         1.1.2 
✔ modeldata    1.3.0      ✔ workflows    1.1.4 
✔ parsnip      1.2.0      ✔ workflowsets 1.0.1 
✔ purrr        1.0.2      ✔ yardstick    1.3.0 
✔ recipes      1.0.10     

── Conflicts ───────────────────────────────────────────────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::

In [2]:
articles <- read_csv('data/OnlineNewsPopularity.csv')

Rows: 39644 Columns: 61
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): url
dbl (60): timedelta, n_tokens_title, n_tokens_content, n_unique_tokens, n_no...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [13]:
articles_temp <- articles

drop_columns <- c('url', 'weekday_is_sunday', 'is_weekend', 'articles_temp.shares')

num_columns <- c('n_tokens_title', 'n_tokens_content', 'num_hrefs','num_self_hrefs',
                 'num_imgs','num_videos','average_token_length','num_keywords','kw_min_min','kw_max_min','kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess')

one_hot_columns <- c('data_channel_is_lifestyle','data_channel_is_entertainment','data_channel_is_bus','data_channel_is_socmed','data_channel_is_tech','data_channel_is_world','weekday_is_monday','weekday_is_tuesday','weekday_is_wednesday','weekday_is_thursday','weekday_is_friday','weekday_is_saturday')

passthrough_columns <- c('n_unique_tokens','n_non_stop_words','n_non_stop_unique_tokens','LDA_00','LDA_01','LDA_02','LDA_03','LDA_04','global_subjectivity','global_sentiment_polarity','global_rate_positive_words','global_rate_negative_words','rate_positive_words','rate_negative_words','avg_positive_polarity','min_positive_polarity','max_positive_polarity','avg_negative_polarity','min_negative_polarity','max_negative_polarity','title_subjectivity','title_sentiment_polarity','abs_title_subjectivity','abs_title_sentiment_polarity')

# One hot encoding
articles_temp[one_hot_columns] <- lapply(articles_temp[one_hot_columns], as.factor)
a <- paste(" shares ~ ",paste(one_hot_columns, collapse=" + "),sep = " ")
dmy <- dummyVars(a, data = articles_temp)
trsf <- data.frame(predict(dmy, newdata = articles_temp), articles_temp$shares)

# trsf <- trsf |> rename(shares = articles_temp.shares)
articles_temp <- cbind(articles_temp, trsf)

# Drop columns
articles_temp <- articles_temp[, !(names(articles_temp) %in% c(drop_columns, one_hot_columns))]

#Log scaling
for(value in num_columns){
    articles_temp[value][articles_temp[value] == 0] = 1
}
articles_temp[articles_temp == -1] <- NA
articles_temp <- na.omit(articles_temp)

articles_temp[num_columns] <- log(articles_temp[num_columns])

articles_temp <- articles_temp |> relocate(shares, .after = last_col())

articles_clean <- articles_temp

In [26]:
set.seed(2024)

articles_clean$ID <- 1:nrow(articles_clean)

training_articles <- sample_n(articles_clean, size = nrow(articles_clean) * 0.70, replace = FALSE)

testing_articles <- anti_join(articles_clean, training_articles, by = "ID")


training_articles <- training_articles[, !(names(training_articles) %in% c('ID'))]
testing_articles <- testing_articles[, !(names(testing_articles) %in% c('ID'))]

head(training_articles, 3)
nrow(training_articles)

head(testing_articles, 3)
nrow(testing_articles)

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,⋯,weekday_is_tuesday.1,weekday_is_wednesday.0,weekday_is_wednesday.1,weekday_is_thursday.0,weekday_is_thursday.1,weekday_is_friday.0,weekday_is_friday.1,weekday_is_saturday.0,weekday_is_saturday.1,shares
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6375,614,2.302585,4.787492,0.7500000,1,0.8734177,1.9459101,1.609438,0,0.000000,⋯,0,1,0,1,0,1,0,0,1,2400
5180,638,2.302585,5.056246,0.7828947,1,0.8514851,0.6931472,0.000000,0,1.386294,⋯,0,0,1,1,0,1,0,1,0,1200
13273,476,2.397895,5.468060,0.6567797,1,0.8014706,1.7917595,1.098612,0,0.000000,⋯,0,1,0,0,1,1,0,1,0,716


[1] 10283

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,⋯,weekday_is_tuesday.1,weekday_is_wednesday.0,weekday_is_wednesday.1,weekday_is_thursday.0,weekday_is_thursday.1,weekday_is_friday.0,weekday_is_friday.1,weekday_is_saturday.0,weekday_is_saturday.1,shares
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,731,2.484907,5.389072,0.6635945,1,0.8153846,1.386294,0.6931472,0,0,⋯,0,1,0,1,0,1,0,1,0,593
2,731,2.197225,5.541264,0.6047431,1,0.7919463,1.098612,0.0000000,0,0,⋯,0,1,0,1,0,1,0,1,0,711
4,731,2.197225,6.274762,0.5037879,1,0.6656347,2.197225,0.0000000,0,0,⋯,0,1,0,1,0,1,0,1,0,1200


[1] 4407

Going to fit an initial linear model using all attributes and ordinary least squares(OLS) to receive a baseline model

In [28]:
article_full_OLS <- lm(shares ~ ., data = training_articles)

tidy(article_full_OLS)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-16352.31164,5354.656477,-3.05384887,0.002265068
timedelta,-1.36129,2.669392,-0.50996246,0.610088769
n_tokens_title,1081.87071,634.125283,1.70608354,0.088022893
n_tokens_content,-834.86008,602.519441,-1.38561517,0.165894663
n_unique_tokens,-9579.74739,5841.487159,-1.63995009,0.101046303
n_non_stop_words,-4893.91077,10762.599043,-0.45471459,0.649324238
n_non_stop_unique_tokens,6300.05917,4007.111463,1.57221960,0.115930570
num_hrefs,446.93644,241.547716,1.85030289,0.064298731
num_self_hrefs,-765.89259,267.360651,-2.86464215,0.004183311


In [29]:
articles_test_pred_full_OLS <- predict(article_full_OLS, newdata = testing_articles)
head(articles_test_pred_full_OLS)

Warning message in predict.lm(article_full_OLS, newdata = testing_articles):
"prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases"


1         2         4        12        20        21 
 468.0467  359.8884 -556.0106  510.6617 -931.3490  725.1515

In [30]:
articles_RMSE_models <- tibble(
    Model = 'OLS Full Regression',
    RMSE = RMSE(
        articles_test_pred_full_OLS,
        testing_articles$shares
    )
)
articles_RMSE_models

Model,RMSE
<chr>,<dbl>
OLS Full Regression,7863.675


Since we have multiple attributes, we wish to find the most significant attributes that contribute the most to the amount of shares an article receives. We will do so using forward selection by way of the `tidymodels` package.

In [34]:
articles_forward_sel <- regsubsets(
    x = shares ~ ., nvmax = 70,
    data = training_articles,
    method = 'forward' 
)

articles_forward_summary <- summary(articles_forward_sel)
articles_forward_summary

Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
"13  linear dependencies found"


Reordering variables and trying again:


Warning message in rval$lopt[] <- rval$vorder[rval$lopt]:
"number of items to replace is not a multiple of replacement length"


Subset selection object
Call: regsubsets.formula(x = shares ~ ., nvmax = 70, data = training_articles, 
    method = "forward")
69 Variables  (and intercept)
                                Forced in Forced out
timedelta                           FALSE      FALSE
n_tokens_title                      FALSE      FALSE
n_tokens_content                    FALSE      FALSE
n_unique_tokens                     FALSE      FALSE
n_non_stop_words                    FALSE      FALSE
n_non_stop_unique_tokens            FALSE      FALSE
num_hrefs                           FALSE      FALSE
num_self_hrefs                      FALSE      FALSE
num_imgs                            FALSE      FALSE
num_videos                          FALSE      FALSE
average_token_length                FALSE      FALSE
num_keywords                        FALSE      FALSE
kw_min_min                          FALSE      FALSE
kw_max_min                          FALSE      FALSE
kw_avg_min                          FALSE      

In [33]:
articles_forward_summary_df <- tibble(
    n_input_variables = 1:56,
    RSQ = articles_forward_summary$rsq,
    RSS = articles_forward_summary$rss,
    ADJ.R2 = articles_forward_summary$adjr2,
    Cp = articles_forward_summary$cp,
    BIC = articles_forward_summary$bic,
)
articles_forward_summary_df

n_input_variables,RSQ,RSS,ADJ.R2,Cp,BIC
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.004974862,1.712065e+12,0.004878079,66.2194946,-32.8076834
2,0.007007660,1.708567e+12,0.006814471,47.0846085,-44.5986984
3,0.008076712,1.706727e+12,0.007787212,37.9697425,-46.4370491
4,0.009110398,1.704949e+12,0.008724762,29.2225676,-47.9203326
5,0.009825568,1.703718e+12,0.009343825,23.7869791,-46.1064795
6,0.010443837,1.702655e+12,0.009866050,19.3588731,-43.2909838
7,0.010822506,1.702003e+12,0.010148614,17.4218729,-37.9884391
8,0.011468664,1.700891e+12,0.010698929,12.7038095,-35.4695212
9,0.011919110,1.700116e+12,0.011053469,10.0205460,-30.9180186


Since we are doing predictive modelling, we will use Mallow's $C_p$ to assess the performance of the different predictive models, where the smaller the $C_p$ value is, the more precise the model relatively is. In this case, using 22 input variables has the smallest $C_p$ value.

The 22 input variables are:
```
n_tokens_title
num_hrefs
num_self_hrefs
num_imgs
kw_max_avg
self_reference_min_shares
self_reference_avg_sharess
LDA_00
LDA_01
LDA_03
global_subjectivity
min_positive_polarity
abs_title_subjectivity
abs_title_sentiment_polarity
data_channel_is_lifestyle.1
data_channel_is_entertainment.1
data_channel_is_bus.0
data_channel_is_socmed.0
data_channel_is_tech.0
data_channel_is_world.1
weekday_is_monday.0
weekday_is_saturday.0

```

Now a reduced model is made using these 22 input variables.

In [37]:
articles_red_OLS <- lm(shares ~ n_tokens_title
+num_hrefs
+num_self_hrefs
+num_imgs
+kw_max_avg
+self_reference_min_shares
+self_reference_avg_sharess
+LDA_00
+LDA_01
+LDA_03
+global_subjectivity
+min_positive_polarity
+abs_title_subjectivity
+abs_title_sentiment_polarity
+data_channel_is_lifestyle.1
+data_channel_is_entertainment.1
+data_channel_is_bus.0
+data_channel_is_socmed.0
+data_channel_is_tech.0
+data_channel_is_world.1
+weekday_is_monday.0
+weekday_is_saturday.0,
        data = training_articles
    )

articles_test_pred_red_OLS <- predict(articles_red_OLS, newdata = testing_articles)

In [38]:
articles_RMSE_models <- rbind(
    articles_RMSE_models,
    tibble(
        Model = "OLS Reduced Regression",
        RMSE = RMSE(
            articles_test_pred_red_OLS,
            testing_articles$shares
        )
    )
)
articles_RMSE_models

Model,RMSE
<chr>,<dbl>
OLS Full Regression,7863.675
OLS Reduced Regression,7851.777


Will also use a K-nearest neighbours model to predict shares using `tidymodels` and 5-fold cross validation to choose the apporiate amount of neighbours using minimum RMSPE.

In [44]:
articles_recipe <- recipe(shares ~ ., data = training_articles) %>%
        step_scale(all_predictors()) %>%
        step_center(all_predictors())

articles_spec <- nearest_neighbor(weight_func = 'rectangular',
                                  neighbors = tune()) %>%
    set_engine('kknn') %>%
    set_mode('regression')

articles_vfold <- vfold_cv(training_articles, v = 3, strata = shares)



In [45]:
gridvals <- tibble(neighbors = seq(1, 30))

articles_multi <- workflow() |>
  add_recipe(articles_recipe) |>
  add_model(articles_spec) |>
  tune_grid(articles_vfold, grid = gridvals) |>
  collect_metrics() |>
  filter(.metric == "rmse") |>
  filter(mean == min(mean))

articles_k <- articles_multi |>
              pull(neighbors)
articles_multi

neighbors,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
30,rmse,standard,12699.09,3,2501.455,Preprocessor1_Model30


We can compare the mean RMSE from K-Nearest Neighbours to the OLS models from earlier, and see that the K-Nearest Neighbours had a larger RMSE, meaning it performs worse in this scenario.

In [46]:
rf <- randomForest(shares ~., data = training_articles, proximity = TRUE)
RMSE(predict(rf, testing_articles), testing_articles$shares)

[1] 8071.475